In [ ]:
import pandas as pd
import numpy as np
from IPython import display
from matplotlib import image
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import ipywidgets as widgets
import plotly.graph_objects as go
import base64
import plotly.express as px
import copy
from docx import Document
from tensorflow import keras
%matplotlib notebook
from ipywidgets import *
import matplotlib.pyplot as pltp
%matplotlib inline

In [ ]:
def classify_logs(df, model):
    
    class_legend = {
    0:"Misc",
    1:'Other BPD',
    2:'Homeless',
    3:'Medical',
    4:'Patron interference',
    5:"Vehicle failure",
    6:"Wayside equipment",
    7:"Software related failures",
    8:"Human Error",
    9:"Weather",
    10:"Info (no error)",
    11:"Delays",
    12:"Track obstruction",
    13:"Schedule maintenance"
    }
    
    df['Cat_Pred'] = df['Log'].apply(lambda log: np.argmax(model.predict([log]), axis=1)[0])
    df['Class_Pred'] = df['Cat_Pred'].apply(lambda code: class_legend[code])
    
    return df
    

In [ ]:
def add_locations(df):
    stationlocs = pd.read_csv("assets/BARTLocationPercentages.csv")
    station_list = (stationlocs['Station_Name']).tolist()
    s_abbrv = [station[:3] for station in station_list]
    testlog = df.head(60)
    
    def lookfornames(c):
        for s in s_abbrv:
            if s in c:
                return s

    stationlocs['Location'] = testlog["Log"].apply(lookfornames)
    testlog['Location'] = testlog["Log"].apply(lookfornames)
    
    justxandy = stationlocs.drop(columns=['Station_Name', 'Rain_Critical', 'Asset_Location'])
    merged = testlog.merge(justxandy.set_index('Location'), on='Location')
    merged['dupped'] = merged.duplicated(subset=['Log'])
    merged = merged[merged.dupped != True]
    vizdata = merged.drop(columns=['dupped'])
    return vizdata
    

In [ ]:
def word_doc_to_df(filename):
    wordDoc = Document(filename)
    dfs = []
    for table in wordDoc.tables:
        data = [[cell.text for cell in row.cells] for row in table.rows]
        dfs.append(pd.DataFrame(data))
    #column labels are the values of the first row
    df = pd.concat(dfs, ignore_index=True)
    df.columns = df.iloc[0] 
    df = df[1:]
    df = df[pd.to_numeric(df['Time'], errors='coerce').notnull()]
    df['Time'] = pd.to_numeric(df["Time"])
    df = df[df['Log'].map(len) < 300]
    return df

In [ ]:
#function for plotly geo

def geoplotly(df):
    df = add_locations(df)
    model = keras.models.load_model('occ-log-classification/model')
    vizdata = classify_logs(df, model)
    maptable = copy.copy(vizdata)
    maptable["xplot"] = (1200* (maptable['X_Percentage']/100))
    maptable["yplot"] = (1080* (maptable['Y_Percentage']/100))
    frametable = maptable[["xplot", "yplot"]]
    xvals = maptable['xplot']
    yvals = maptable['yplot']
    
    #graph
    mapfig = go.Figure()

    img_width = 1200
    img_height = 1080
    mapfig.update_layout(
        autosize=False
    )

    bartimage = base64.b64encode(open("assets/BARTtracksmap.png", 'rb').read())
    mapfig.add_layout_image(
            dict(
                source='data:image/png;base64,{}'.format(bartimage.decode()), #"./assets/BARTtracksmap.png",
                xref="x",
                yref="y",
                x=0,
                y=0,
                sizex=img_width,
                sizey=img_height,
                #sizing="stretch",
                opacity=1,
                sizing='contain',
                layer="below",xanchor="left", yanchor="top")
    )
    mapfig.update_xaxes(showgrid=False, range=(0, img_width))
    mapfig.update_yaxes(showgrid=False, scaleanchor='x', range=(img_height, 0))

    mapfig.add_trace(
        go.Scatter(x=maptable["xplot"], y=maptable["yplot"], mode="markers", text=maptable['Log'])
    )
    
    #swimscatter
    catvizdata = vizdata.drop(columns=['X_Percentage', 'Y_Percentage'])
    refs = np.random.choice(np.arange(len(catvizdata['Location'])), len(catvizdata['Location']), replace=False)
    catvizdata['Reference'] = refs
    import datetime
    timarr = []
    for t in catvizdata['Time']:
        t = int(t)
        temp = datetime.time(t//100, t%100)
        timarr.append(temp)
    catvizdata['Time'] = timarr
    from datetime import time
    timeaxis = []
    start = datetime.time(5, 0, 0)
    end = datetime.time(22, 0, 0)
    currhr = 5
    currmin = 1
    timeaxis.append(start)
    for i in range(1020):
        nexttime = datetime.time(currhr, currmin, 0)
        timeaxis.append(nexttime)
        if currmin == 59:
            currmin = 0
            currhr = currhr + 1
        else:
            currmin = currmin + 1
    catvizdata = catvizdata.sort_values(by=['Time'])
    xaxistime = catvizdata['Time']
    swimscatter = px.scatter(catvizdata, y="Class_Pred", x='Time', color="Class_Pred", symbol="Class_Pred", hover_data=['Log', 'Location'])
    swimscatter.update_traces(marker_size=20)
    swimscatter.update_xaxes(categoryorder='category ascending', showgrid=False)
    
    record = catvizdata[["Time", "Log", "Class_Pred"]]
    
   # return mapfig, swimscatter
    return html.Div([
        dcc.Graph(figure=mapfig),
        dcc.Graph(figure=swimscatter),
        dash_table.DataTable(
            record.to_dict('records'),
            [{'name': i, 'id': i} for i in record.columns]
        )
    ])




    

In [ ]:
import base64
import os
from urllib.parse import quote as urlquote

from flask import Flask, send_from_directory
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output


UPLOAD_DIRECTORY = "/project/app_uploaded_files"

if not os.path.exists(UPLOAD_DIRECTORY):
    os.makedirs(UPLOAD_DIRECTORY)


# Normally, Dash creates its own Flask server internally. By creating our own,
# we can create a route for downloading files directly:
server = Flask(__name__)
app = dash.Dash(server=server)


@server.route("/download/<path:path>")
def download(path):
    """Serve a file from the upload directory."""
    return send_from_directory(UPLOAD_DIRECTORY, path, as_attachment=True)

app.layout = html.Div(
    [
        html.H1("OCC Dashboard"),
        html.H2("Upload Logs"),
        dcc.Upload(
            id="upload-data",
            children=html.Div(
                ["Drag and drop or click to select a file to upload."]
            ),
            style={
                "width": "100%",
                "height": "60px",
                "lineHeight": "60px",
                "borderWidth": "1px",
                "borderStyle": "dashed",
                "borderRadius": "5px",
                "textAlign": "center",
                "margin": "10px",
            },
            multiple=True,
        ),
        html.H2("Dashboard"),
        html.Ul(id="file-list"),
    ],
)

def save_file(name, content):
    """Decode and store a file uploaded with Plotly Dash."""
    data = content.encode("utf8").split(b";base64,")[1]
    with open(os.path.join(UPLOAD_DIRECTORY, name), "wb") as fp:
        fp.write(base64.decodebytes(data))


def uploaded_files():
    """List the files in the upload directory."""
    files = []
    for filename in os.listdir(UPLOAD_DIRECTORY):
        path = os.path.join(UPLOAD_DIRECTORY, filename)
        if os.path.isfile(path):
            files.append(filename)
    return files


def file_download_link(filename):
    """Create a Plotly Dash 'A' element that downloads a file from the app."""
    location = "/download/{}".format(urlquote(filename))
    return html.A(filename, href=location)


@app.callback(
    Output("file-list", "children"),
    [Input("upload-data", "filename"), Input("upload-data", "contents")],
)
def update_output(uploaded_filenames, uploaded_file_contents):
    """Save uploaded files and regenerate the file list."""

    if uploaded_filenames is not None and uploaded_file_contents is not None:
        for name, data in zip(uploaded_filenames, uploaded_file_contents):
            save_file(name, data)

    files = uploaded_files()
    if len(files) == 0:
        return [html.Li("No files yet!")]
    else:
        return [geoplotly(word_doc_to_df(os.path.join(UPLOAD_DIRECTORY, filename))) for filename in files]

if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)